In [1]:
import matplotlib.pyplot as plt
import torch
from torch.quasirandom import SobolEngine
import pandas as pd
import numpy as np
from alse.utils import identify_samples_which_satisfy_constraints, normalize, unnormalize
from mpl_toolkits import mplot3d
# from matplotlib import cm
from alse.utils import read_excel
from alse.alse_workflow import alse
import copy
import plotly.graph_objects as go
import plotly.express as px

In [2]:
bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}
file_path = "../test_data/8_12_data.xlsx"
ts = 20
X, width, pow_cap, wth = read_excel("../test_data/8_12_data.xlsx",["V (mm/min)", "P (W)", "spot size (mm)"], ["width (mm)", "powder_cap", "wth"])
X=X[:ts, :]
width=width[:ts]
pow_cap=pow_cap[:ts]
wth=wth[:ts]
output_param = [width, pow_cap, wth]
bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
constraints = [("gt", 2.5), ("gt", 50), ("gt", 6)]

algo = alse(X, bounds, output_param, constraints)

algo.initialize_model(["reg", "reg", "reg"])

test_file_path = "../test_data/3d_grid.xlsx"
X_test, width_test, pow_cap_test, wth_test = read_excel(test_file_path,["V (mm/min)", "P (W)", "Spot (mm)"], ["widths avg (mm)", "powder capt %", "w/h"])

X_test_normalize = normalize(X_test, bounds)
for model in algo.list_of_models:
    model.eval()
model_prediction = [
    model(X_test_normalize.float()).loc.detach() for model in algo.list_of_models
]
al_pred, al_pred_overlap = algo.get_points_mask(model_prediction)
grid, grid_overlap = algo.get_points_mask([width_test, pow_cap_test, wth_test])
print(al_pred[0] == grid[0])
print(al_pred[1] == grid[1])
print(al_pred[2] == grid[2])
print(sum(grid_overlap == al_pred_overlap))
print(grid_overlap)

/opt/conda/lib/python3.9/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:136: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2189.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True, False, False,  True,  True,  True,  True,  True])
tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False, False,  True,  True,  True,  True,  True,  True,  True])
tensor([False,  True,  True,  True,  True, False,  True,  True,  True,  True,
        False, False,  True,  True,  True,  True,  True,  True,  True,  True])
tensor(17)
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True,  True,  True,  True, False])


In [3]:
bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}
file_path = "../test_data/8_12_data.xlsx"
test_file_path = "../test_data/3d_grid.xlsx"

X, width, pow_cap, wth = read_excel(test_file_path,["V (mm/min)", "P (W)", "Spot (mm)"], ["widths avg (mm)", "powder capt %", "w/h"])
X_test, width_test, pow_cap_test, wth_test = read_excel(file_path,["V (mm/min)", "P (W)", "spot size (mm)"], ["width (mm)", "powder_cap", "wth"])

output_param = [width, pow_cap, wth]
bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
constraints = [("gt", 2.5), ("gt", 50), ("gt", 6)]

algo_grid = alse(X, bounds, output_param, constraints)

algo_grid.initialize_model(["reg", "reg", "reg"])


X_test_normalize = normalize(X_test, bounds)
for model in algo_grid.list_of_models:
    model.eval()
model_prediction = [
    model(X_test_normalize.float()).loc.detach() for model in algo.list_of_models
]
al_pred, al_pred_overlap = algo_grid.get_points_mask(model_prediction)
grid, grid_overlap = algo_grid.get_points_mask([width_test, pow_cap_test, wth_test])
print(al_pred[0] == grid[0])
print(al_pred[1] == grid[1])
print(al_pred[2] == grid[2])
print(sum(grid_overlap == al_pred_overlap))
print(grid_overlap)

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])
tensor(20)
tensor([False, False,  True, False, False,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True])


/opt/conda/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [7]:
N=20
pred, overlap = algo.get_posterior_grid(N)
# minus_15, plus_15 = algo.get_test_points(num_points=10, uncertainty=0.15)

x=np.linspace(bounds[0][0], bounds[1][0], N)
y=np.linspace(bounds[0][1], bounds[1][1], N)
z=np.linspace(bounds[0][2], bounds[1][2], N)

Xplt, Yplt, Zplt = np.meshgrid(x,y,z)
Xplt = torch.from_numpy(Xplt)
Yplt = torch.from_numpy(Yplt)
Zplt = torch.from_numpy(Zplt)

fig = go.Figure(
    go.Isosurface(
        x=Xplt.flatten(),
        y=Yplt.flatten(),
        z=Zplt.flatten(),
        value=overlap,
        isomin=0,
        isomax=1,
        surface_count=4,  # number of isosurfaces, 2 by default: only min and max
        colorbar_nticks=4,  # colorbar ticks correspond to isosurface values
        caps=dict(x_show=False, y_show=False),
    )
)
fig.add_trace(
    go.Scatter3d(
        x=X[:,0],
        y=X[:,1],
        z=X[:,2],
        # text=[f"{yname[i]} : {value}" for value in dff[yname[i]]],
        mode="markers",
        marker=dict(size=5, color='black'),
        name = "current points"
    )
)
# fig.add_trace(
#     go.Scatter3d(
#         x=plus_15[:,0],
#         y=plus_15[:,1],
#         z=plus_15[:,2],
#         # text=[f"{yname[i]} : {value}" for value in dff[yname[i]]],
#         mode="markers",
#     )
# )
fig.update_layout(
    autosize=True,
    showlegend=True,
    scene=dict(
        xaxis_title="Scan velocity (mm/min)", yaxis_title="Laser power (W)", zaxis_title="Spot size (mm)"
    ),
    margin=dict(l=0, r=0, b=0, t=0),
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

In [ ]:
N=20
pred, overlap = algo_grid.get_posterior_grid(N)
# minus_15, plus_15 = algo.get_test_points(num_points=10, uncertainty=0.15)

x=np.linspace(bounds[0][0], bounds[1][0], N)
y=np.linspace(bounds[0][1], bounds[1][1], N)
z=np.linspace(bounds[0][2], bounds[1][2], N)

Xplt, Yplt, Zplt = np.meshgrid(x,y,z)
Xplt = torch.from_numpy(Xplt)
Yplt = torch.from_numpy(Yplt)
Zplt = torch.from_numpy(Zplt)

fig = go.Figure(
    go.Isosurface(
        x=Xplt.flatten(),
        y=Yplt.flatten(),
        z=Zplt.flatten(),
        value=overlap,
        isomin=0,
        isomax=1,
        surface_count=4,  # number of isosurfaces, 2 by default: only min and max
        colorbar_nticks=4,  # colorbar ticks correspond to isosurface values
        caps=dict(x_show=False, y_show=False),
    )
)
fig.add_trace(
    go.Scatter3d(
        x=X_test[:,0],
        y=X_test[:,1],
        z=X_test[:,2],
        # text=[f"{yname[i]} : {value}" for value in dff[yname[i]]],
        mode="markers",
        marker=dict(size=5, color='black')
    )
)
# fig.add_trace(
#     go.Scatter3d(
#         x=plus_15[:,0],
#         y=plus_15[:,1],
#         z=plus_15[:,2],
#         # text=[f"{yname[i]} : {value}" for value in dff[yname[i]]],
#         mode="markers",
#     )
# )
fig.update_layout(
    autosize=True,
    scene=dict(
        xaxis_title="Scan velocity (mm/min)", yaxis_title="Laser power (W)", zaxis_title="Spot size (mm)"
    ),
    margin=dict(l=0, r=0, b=0, t=0),
)
fig.show()

In [ ]:
print(minus_15)